In [1]:
# export JT_SAVE_MEM=1
# # 限制cpu最多使用16G
# export cpu_mem_limit=-1
# # 限制device内存（如gpu、tpu等）最多使用8G
# export gpu_mem_limit=15000000000
# import os
# os.environ["JT_SAVE_MEM"] = '1'
# os.environ["cpu_mem_limit"] = '-1'
# os.environ["gpu_mem_limit"] = '15000000000'
import jclip as clip
from jittor.dataset import Dataset
from jittor.dataset import DataLoader
from PIL import Image
import jittor as jt
import jittor.nn as nn
import jittor.optim as optim
from jittor import transform, utils
import os
import time
import numpy as np
from tqdm import tqdm
import pandas as pd

[i 0801 15:11:31.291198 88 lock.py:85] Create lock file:/root/.cache/jittor/jt1.3.9/g++11.4.0/py3.10.14/Linux-4.19.24-xe6/IntelRXeonRPlaxf9/d731/jittor.lock
[i 0801 15:11:31.327026 88 compiler.py:956] Jittor(1.3.9.5) src: /usr/local/lib/python3.10/site-packages/jittor
[i 0801 15:11:31.332849 88 compiler.py:957] g++ at /usr/bin/g++(11.4.0)
[i 0801 15:11:31.332957 88 compiler.py:958] cache_path: /root/.cache/jittor/jt1.3.9/g++11.4.0/py3.10.14/Linux-4.19.24-xe6/IntelRXeonRPlaxf9/d731/default
[i 0801 15:11:31.349484 88 __init__.py:412] Found nvcc(12.1.105) at /usr/local/cuda/bin/nvcc.
[i 0801 15:11:31.783860 88 __init__.py:412] Found gdb(22.04.2) at /usr/bin/gdb.
[i 0801 15:11:31.790569 88 __init__.py:412] Found addr2line(2.38) at /usr/bin/addr2line.
[i 0801 15:11:32.178259 88 compiler.py:1011] cuda key:cu12.1.105_sm_70
[i 0801 15:11:32.204974 88 compiler.py:34] Create cache dir: /root/.cache/jittor/jt1.3.9/g++11.4.0/py3.10.14/Linux-4.19.24-xe6/IntelRXeonRPlaxf9/d731/default/cu12.1.105_sm_

[i 0801 15:11:39.506002 08 log.cc:351] Load log_sync: 1
[i 0801 15:11:54.833640 08 __init__.py:227] Total mem: 31.41GB, using 10 procs for compiling.


Compiling jittor_core(151/151) used: 61.306s eta: 0.000ss


[i 0801 15:12:56.574031 08 log.cc:351] Load log_sync: 1
[i 0801 15:12:56.575400 08 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0801 15:12:56.709734 08 init.cc:63] Found cuda archs: [70,]
[i 0801 15:12:59.269611 08 __init__.py:412] Found mpicc(4.1.2) at /usr/bin/mpicc.
[i 0801 15:12:59.283317 08 compiler.py:34] Create cache dir: /root/.cache/jittor/jt1.3.9/g++11.4.0/py3.10.14/Linux-4.19.24-xe6/IntelRXeonRPlaxf9/d731/default/cu12.1.105_sm_70/custom_ops


[i 0801 15:13:02.142047 08 compile_extern.py:378] Downloading cutt...
[i 0801 15:13:02.153141 08 compile_extern.py:391] installing cutt...


Compiling jittor_mpi_core(7/7) used: 2.752s eta: 0.000s
Compiling libcutt(9/9) used: 12.662s eta: 0.000s


[i 0801 15:13:21.500549 08 compiler.py:34] Create cache dir: /root/.cache/jittor/jt1.3.9/g++11.4.0/py3.10.14/Linux-4.19.24-xe6/IntelRXeonRPlaxf9/d731/default/cu12.1.105_sm_70/cuda


Compiling gen_ops_mkl_test_mkl_conv_mkl_matmul_mkl_conv_back___hashcad247(7/7) used: 4.538s eta: 0.000s
Compiling gen_ops_cub_test_cub_cumsum_cub_argsort_cub_where____hash4eb589(6/6) used: 2.396s eta: 0.000s
Compiling gen_ops_cublas_test_cublas_acc_matmul_cublas_matmu___hash02a226(8/8) used: 2.636s eta: 0.000s
Compiling gen_ops_cudnn_conv_backward_w_cudnn_conv3d_backwar___hash25ad86(16/16) used: 8.139s eta: 0.000s


In [2]:
# load data
class TripleDataset(Dataset):
    def __init__(self, img_path, img_label, img_description, transform=None):
        super(TripleDataset, self).__init__()
        self.img_path = img_path
        self.img_label = img_label
        self.img_description = img_description 
        
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        
        img = Image.open(self.img_path[index]).convert('RGB')
        label = self.img_label[index]
        img_description = self.img_description[index]
        
        if self.transform is not None:
            img = self.transform(img)
            
        return img, label, img_description

    def __len__(self):
        return len(self.img_path)

In [3]:
train_label = pd.read_csv('/mnt/workspace/JCLIP/Dataset/train_1234_ft.txt')
val_label = pd.read_csv('/mnt/workspace/JCLIP/Dataset/valid.txt')
test_label = pd.read_csv('/mnt/workspace/JCLIP/Dataset/test.txt')

In [4]:
train_label['path'] = '/mnt/workspace/JCLIP/Dataset/' + train_label['img_name']
val_label['path'] = '/mnt/workspace/JCLIP/Dataset/' + val_label['img_name']
test_label['path'] = '/mnt/workspace/JCLIP/Dataset/TestSetA/' + test_label['img_name']
train_label['description'] = None
val_label['description'] = None
test_label['description'] = None

In [5]:
def append_description(dataframe):
    for index, row in dataframe.iterrows():
        paths = row['img_name'].split('/')
        # test
        if len(paths) >= 2:
            cname = paths[-2]
            cname_father = paths[-3]
            if cname_father.startswith('Animal'):
                dataframe.at[index, 'description'] = 'A photo of ' + cname.lower().replace('_', ' ') + ', a type of animal'
            elif cname_father.startswith('Thu-dog'):
                dataframe.at[index, 'description'] = 'A photo of ' + cname.lower().replace('_', ' ') + ', a type of dog'
            elif cname_father.startswith('Caltech-101'):
                dataframe.at[index, 'description'] = 'A photo of ' + cname.lower().replace('_', ' ')
            elif cname_father.startswith('Food-101'):
                dataframe.at[index, 'description'] = 'A photo of ' + cname.lower().replace('_', ' ') + ', a type of food'
        else:
            return dataframe
    return dataframe

In [6]:
train_label = append_description(train_label)
val_label = append_description(val_label)
test_label = append_description(test_label)

In [7]:
test_label

,img_name,path,description
0,image_855.jpeg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_85...,None
1,image_2480.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_24...,None
2,image_1008.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_10...,None
3,image_1230.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_12...,None
4,image_1229.jpeg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_12...,None
...,...,...,...
3068,image_2225.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_22...,None
3069,image_896.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_89...,None
3070,image_323.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_32...,None
3071,image_1663.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_16...,None


In [8]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = ""

    def pr2int(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [9]:
def validate(val_loader, model, preprocess, loss_img, text_features, bs_value):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    batch_num = int(len(val_loader) / bs_value)
    progress = ProgressMeter(batch_num, batch_time, losses, top1)


    with jt.no_grad():
        end = time.time()
        for i, (imgs, target, description) in tqdm(enumerate(val_loader), total=batch_num):
            # img [1, 224, 224, 3]  但是用不着

            # 将 NumPy 数组转换为 PIL 图像并进行预处理
            images = [Image.fromarray(img_np) for img_np in imgs.numpy()]


            # 预处理图像
            images_preprocessed = jt.cat([preprocess(image).unsqueeze(0) for image in images], dim=0)
            images_preprocessed = jt.misc.to(images_preprocessed, 'cuda')
            
            images_clip_features = model.encode_image(images_preprocessed)
            # [batch_size, 512]
            images_clip_features /= images_clip_features.norm(dim=-1, keepdim=True)

            target = jt.misc.to(target, 'cuda')

            # compute output
            text_probs = (100.0 *
                  images_clip_features @ text_features.transpose(0, 1)).softmax(
                      dim=-1)

            loss = loss_img(text_probs, target)
            # measure accuracy and record loss
            losses.update(loss.item(), imgs.size(0))

            # measure accuracy and record loss
            acc = (text_probs.argmax(1)[0].view(-1) == target.float().view(-1)).float().mean() * 100
            top1.update(acc, n=imgs.size(0))
            
            
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f}'
              .format(top1=top1))

        return top1


def predict(test_label, model, preprocess, text_features, tta=10):


    test_num = len(test_label)

    test_pred_tta = None
    for _ in range(tta):
        test_pred = []
        with jt.no_grad():
            end = time.time()
            for i in tqdm(range(test_num), total=test_num):

                img_path = test_label['path'][i]
                image = Image.open(img_path)
                image = preprocess(image).unsqueeze(0)

                image_features = model.encode_image(image)
                image_features /= image_features.norm(dim=-1, keepdim=True)

                # compute output
                output = (100.0 *
                      image_features @ text_features.transpose(0, 1)).softmax(
                          dim=-1)

                output = nn.softmax(output, dim=1)
                output = output.numpy()

                test_pred.append(output)
        test_pred = np.vstack(test_pred)

        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred

    return test_pred_tta


def train(train_loader, model, preprocess, loss_img, loss_txt, optimizer, bs_value):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top1_text = AverageMeter('Acc@1', ':6.2f')

    # jittor的len(train_loader)不会随着batch_size的变化而改变
    # 永远都是1496
    batch_num = int(len(train_loader) / bs_value)
    progress = ProgressMeter(batch_num, batch_time, losses, top1)

    # switch to train mode
    model.train()

    end = time.time()
    for i, (imgs, target, description) in enumerate(train_loader):
        # img [1, 224, 224, 3]  但是用不着

        # 将 NumPy 数组转换为 PIL 图像并进行预处理
        images = [Image.fromarray(img_np) for img_np in imgs.numpy()]


        # 预处理图像
        images_preprocessed = jt.cat([preprocess(image).unsqueeze(0) for image in images], dim=0)
        images_preprocessed = jt.misc.to(images_preprocessed, 'cuda')

        # 描述
        description = clip.tokenize(description)
        description = jt.misc.to(description, 'cuda')

        logits_per_image, logits_per_text = model(images_preprocessed, description)

        ground_truth = jt.arange(len(images_preprocessed),dtype=jt.float32).cuda()

        loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2
        # measure accuracy and record loss
        losses.update(loss.item(), imgs.size(0))

        # measure accuracy and record loss
        acc = (logits_per_image.argmax(1)[0].view(-1) == ground_truth.float().view(-1)).float().mean() * 100
        acc_text = (logits_per_text.argmax(1)[0].view(-1) == ground_truth.float().view(-1)).float().mean() * 100
        top1.update(acc, n=imgs.size(0))
        top1_text.update(acc_text, n=imgs.size(0))

        optimizer.zero_grad()
        optimizer.backward(loss)
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 16 == 0:
            progress.pr2int(i)
            print(' * Acc_text@1 {top1_text.avg:.3f}'
                  .format(top1_text=top1_text))
    return top1


In [15]:
# parameter
epoch_num = 32
bs_value = 4

In [16]:
train_trans = transform.Compose([
                transform.Resize((224, 224)),
                transform.RandomRotation(degrees=180),
                transform.ColorJitter(0.5),
                transform.RandomHorizontalFlip(),
                transform.RandomVerticalFlip(),
                # transform.ToTensor(),
                # transform.ImageNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
val_trans = transform.Compose([
                transform.Resize((224, 224)),
                # transform.ToTensor(),
                # transform.ImageNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
train_dataset = TripleDataset(train_label['path'], train_label['target'], train_label['description'], train_trans).set_attrs(batch_size=bs_value, shuffle=True)
val_dataset = TripleDataset(val_label['path'], val_label['target'], val_label['description'], val_trans).set_attrs(batch_size=bs_value, shuffle=False)
train_loader = DataLoader(train_dataset)
val_loader = DataLoader(val_dataset)


In [17]:
def process_textfeatures(model_clip):
    # 加载clip


    classes = open('/mnt/workspace/JCLIP/Dataset/classes.txt').read().splitlines()
    # remove the prefix Animal, Thu-dog, Caltech-101, Food-101

    new_classes1 = []
    new_classes2 = []
    for c in classes:
        c = c.split(' ')[0]
        if c.startswith('Animal'):
            c = c[7:]
            c1 = 'A photo of ' + c.lower().replace('_', ' ') + ', a type of animal'
            c2 = 'A photo animal of ' + c.lower().replace('_', ' ') #+ '.'
        if c.startswith('Thu-dog'):
            c = c[8:]
            c1 = 'A photo of ' + c.lower().replace('_', ' ') + ', a type of dog'
            c2 = 'A photo dog of ' + c.lower().replace('_', ' ') #+ '.'
        if c.startswith('Caltech-101'):
            c = c[12:]
            c1 = 'A photo of ' + c.lower().replace('_', ' ')
            c2 = 'A photo of ' + c.lower().replace('_', ' ') #+ '.'
        if c.startswith('Food-101'):
            c = c[9:]
            c1 = 'A photo of ' + c.lower().replace('_', ' ') + ', a type of food'
            c2 = 'A photo food of ' + c.lower().replace('_', ' ') #+ '.'
        new_classes1.append(c1)
        new_classes2.append(c2)

    text1 = clip.tokenize(new_classes1)
    text2 = clip.tokenize(new_classes2)


    text_features1 = model_clip.encode_text(text1)
    text_features1 /= text_features1.norm(dim=-1, keepdim=True)

    text_features2 = model_clip.encode_text(text2)
    text_features2 /= text_features2.norm(dim=-1, keepdim=True)

    text_features = (text_features1 + text_features2) / 2.
    text_features /= text_features.norm(dim=-1, keepdim=True)
    return text_features

# 模型加载

In [18]:
model_path = "../models_pkl/clip_ft/clip_ft_Epoch.pkl"
# 加载clip
model_clip, preprocess = clip.load("/mnt/workspace/JCLIP/ViT-B-32.pkl")
model_clip.cuda()
print('创建完毕')

if os.path.exists(model_path):
    model_clip.load(model_path)
    print("使用以往的模型继续训练")
else:
    print("使用hugging face预训练模型")

创建完毕
使用hugging face预训练模型


In [19]:
# Params used from paper, the lr is smaller, more safe for fine tuning to new dataset
optimizer = optim.Adam(model_clip.parameters(), lr=1e-8,betas=(0.9,0.98),eps=1e-6,weight_decay=0.001)
scheduler = jt.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.85)
# 创建损失函数
loss_img = nn.CrossEntropyLoss().cuda()
loss_txt = nn.CrossEntropyLoss().cuda()


best_acc = 0.0
acc_list = []
for epoch in range(epoch_num):
    scheduler.step()
    print('Epoch: ', epoch)

    train_acc = train(train_loader, model_clip, preprocess, loss_img, loss_txt, optimizer, bs_value)
    # text_features = process_textfeatures(model_clip)
    # val_acc = validate(val_loader, model_clip, preprocess, loss_img, text_features, bs_value)

    # if train_acc.avg.item() > best_acc:
    best_acc = round(train_acc.avg.item(), 2)
    acc_list.append(best_acc)
    if (epoch + 1) % 4 == 0:
        model_clip.save(f'../models_pkl/clip_ft/clip_ft_Epoch{epoch}_{best_acc}.pkl')

Epoch:  0



Compiling Operators(2/6) used: 6.31s eta: 12.6s 4/6) used: 8.32s eta: 4.16s 6/6) used: 9.32s eta:    0s 

Compiling Operators(1/55) used: 8.31s eta:  449s 3/55) used: 9.32s eta:  162s 4/55) used: 10.3s eta:  132s 5/55) used: 11.3s eta:  113s 7/55) used: 12.4s eta: 84.7s 8/55) used: 13.4s eta: 78.5s 9/55) used: 18.4s eta: 93.9s 12/55) used: 21.4s eta: 76.6s 13/55) used: 22.4s eta: 72.3s 15/55) used: 23.4s eta: 62.4s 16/55) used: 25.4s eta: 61.9s 17/55) used: 29.4s eta: 65.8s 18/55) used: 30.4s eta: 62.6s 20/55) used: 31.4s eta:   55s 22/55) used: 32.5s eta: 48.7s 23/55) used: 33.5s eta: 46.6s 24/55) used: 35.5s eta: 45.8s 25/55) used: 36.5s eta: 43.8s 26/55) used: 37.5s eta: 41.8s 27/55) used: 40.5s eta:   42s 29/55) used: 42.5s eta: 38.1s 30/55) used: 44.5s eta: 37.1s 31/55) used: 46.6s eta:   36s 32/55) used: 47.6s eta: 34.2s 33/55) used: 48.6s eta: 32.4s 34/55) used: 51.6s eta: 31.9s 35/55) used: 53.6s eta: 30.6s 36/55) used: 54.6s eta: 28.8s 39/55) used: 55.6s eta: 22.8s 41/55) use

[  0/374]	Time 85.040 (85.040)	Loss 2.9653e-01 (2.9653e-01)	Acc@1 100.00 (100.00)
 * Acc_text@1 75.000
Epoch:  1



Compiling Operators(1/65) used: 8.32s eta:  532s 2/65) used: 11.3s eta:  357s 3/65) used: 12.3s eta:  255s 4/65) used: 13.3s eta:  203s 6/65) used: 14.3s eta:  141s 9/65) used: 16.4s eta:  102s 10/65) used: 21.4s eta:  118s 11/65) used: 23.4s eta:  115s 12/65) used: 24.4s eta:  108s 13/65) used: 25.4s eta:  102s 14/65) used: 26.4s eta: 96.2s 16/65) used: 27.4s eta: 83.9s 19/65) used: 31.4s eta: 76.1s 20/65) used: 33.4s eta: 75.2s 21/65) used: 35.4s eta: 74.2s 22/65) used: 37.5s eta: 73.2s 23/65) used: 38.5s eta: 70.2s 24/65) used: 39.5s eta: 67.4s 25/65) used: 40.5s eta: 64.8s 26/65) used: 43.5s eta: 65.2s 27/65) used: 44.5s eta: 62.6s 30/65) used: 46.5s eta: 54.3s 31/65) used: 47.5s eta: 52.1s 32/65) used: 50.5s eta: 52.1s 33/65) used: 51.6s eta:   50s 34/65) used: 52.6s eta: 47.9s 35/65) used: 53.6s eta: 45.9s 38/65) used: 57.6s eta: 40.9s 39/65) used: 58.6s eta: 39.1s 41/65) used: 62.6s eta: 36.6s 43/65) used: 63.6s eta: 32.5s 44/65) used: 66.6s eta: 31.8s 46/65) used: 67.6s eta: 2

[  0/374]	Time 93.701 (93.701)	Loss 1.2684e-03 (1.2684e-03)	Acc@1 100.00 (100.00)
 * Acc_text@1 100.000
Epoch:  2
[  0/374]	Time  0.138 ( 0.138)	Loss 3.5282e-01 (3.5282e-01)	Acc@1  75.00 ( 75.00)
 * Acc_text@1 100.000
Epoch:  3
[  0/374]	Time  0.147 ( 0.147)	Loss 7.0714e-02 (7.0714e-02)	Acc@1 100.00 (100.00)
 * Acc_text@1 100.000



Compiling Operators(1/3) used: 5.31s eta: 10.6s 3/3) used: 6.32s eta:    0s 

Compiling Operators(1/5) used: 6.31s eta: 25.2s 3/5) used: 7.32s eta: 4.88s 5/5) used: 8.33s eta:    0s 

Compiling Operators(2/2) used: 5.31s eta:    0s 

Compiling Operators(1/1) used: 4.72s eta:    0s 

Compiling Operators(1/1) used: 4.86s eta:    0s 

Compiling Operators(1/1) used: 4.88s eta:    0s 


Epoch:  4
[  0/374]	Time  0.222 ( 0.222)	Loss 5.5615e-04 (5.5615e-04)	Acc@1 100.00 (100.00)
 * Acc_text@1 100.000
Epoch:  5
[  0/374]	Time  0.170 ( 0.170)	Loss 2.5673e-05 (2.5673e-05)	Acc@1 100.00 (100.00)
 * Acc_text@1 100.000
Epoch:  6
[  0/374]	Time  0.139 ( 0.139)	Loss 2.4386e-01 (2.4386e-01)	Acc@1 100.00 (100.00)
 * Acc_text@1 75.000
Epoch:  7
[  0/374]	Time  0.170 ( 0.170)	Loss 5.3864e-03 (5.3864e-03)	Acc@1 100.00 (100.00)
 * Acc_text@1 100.000
Epoch:  8
[  0/374]	Time  0.217 ( 0.217)	Loss 1.5821e-01 (1.5821e-01)	Acc@1  75.00 ( 75.00)
 * Acc_text@1 100.000
Epoch:  9
[  0/374]	Time  0.173 ( 0.173)	Loss 2.1968e-02 (2.1968e-02)	Acc@1 100.00 (100.00)
 * Acc_text@1 100.000
Epoch:  10
[  0/374]	Time  0.190 ( 0.190)	Loss 8.7901e-02 (8.7901e-02)	Acc@1 100.00 (100.00)
 * Acc_text@1 100.000
Epoch:  11
[  0/374]	Time  0.187 ( 0.187)	Loss 9.2138e-01 (9.2138e-01)	Acc@1  50.00 ( 50.00)
 * Acc_text@1 75.000
Epoch:  12
[  0/374]	Time  2.044 ( 2.044)	Loss 2.9004e-02 (2.9004e-02)	Acc@1 100.00 (100

In [ ]:
acc_list

In [31]:
model_path = "../models_pkl/clip_ft/clip_ft_Epoch27_0.0.pkl"
model_clip.load(model_path)

In [20]:
loss_img = nn.CrossEntropyLoss().cuda()
text_features = process_textfeatures(model_clip)
val_acc = validate(val_loader, model_clip, preprocess, loss_img, text_features, bs_value)

    # if train_acc.avg.item() > best_acc:
    # best_acc = round(val_acc.avg.item(), 2)
    # acc_list.append(best_acc)

  0%|          | 0/935 [00:00<?, ?it/s][i 0801 15:30:40.327773 08 cuda_device_allocator.cc:30] 
=== display_memory_info ===
 total_cpu_ram: 31.41GB total_device_ram: 15.78GB
 hold_vars: 1254 lived_vars: 4153 lived_ops: 19359
 name: sfrl is_device: 1 used: 12.99GB(91.3%) unused: 1.236GB(8.68%) ULB: 46.08MB ULBO:   170MB total: 14.23GB
 name: sfrl is_device: 1 used:     0 B(-nan%) unused:     0 B(-nan%) total:     0 B
 name: sfrl is_device: 0 used:     0 B(-nan%) unused:     0 B(-nan%) total:     0 B
 name: sfrl is_device: 0 used: 2.189MB(0.263%) unused: 831.8MB(99.7%) ULB:    98MB ULBO:    98MB total:   834MB
 name: sfrl is_device: 0 used:     0 B(-nan%) unused:     0 B(-nan%) total:     0 B
 name: temp is_device: 0 used:     0 B(-nan%) unused:     0 B(-nan%) total:     0 B
 name: temp is_device: 1 used:     0 B(0%) unused: 9.606MB(100%) total: 9.606MB
 cpu&gpu: 15.05GB gpu: 14.24GB cpu:   834MB
 free: cpu(14.27GB) gpu(205.8MB)
 swap: total(    0 B) last(    0 B)

[i 0801 15:30:40.62911

RuntimeError: [38;5;1m[f 0801 15:30:40.632038 08 executor.cc:686] 
Execute fused operator(5725/6459) failed. 
[JIT Source]: /root/.cache/jittor/jt1.3.9/g++11.4.0/py3.10.14/Linux-4.19.24-xe6/IntelRXeonRPlaxf9/d731/default/cu12.1.105_sm_70/jit/__opkey0_array__T_float32__opkey1_broadcast_to__Tx_float32__DIM_3__BCAST_3__opkey2_broadca___hash_febb7ee8e2a4d734_op.cc 
[OP TYPE]: fused_op:( array, broadcast_to, broadcast_to, binary.add, binary.multiply, unary.sigmoid, binary.multiply,)
[Input]: float32[2048,]transformer.resblocks.10.mlp.c_fc.bias, float32[77,374,2048,], 
[Output]: float32[77,374,2048,], 
[Async Backtrace]: not found, please set env JT_SYNC=1, trace_py_var=3 
[Reason]: [38;5;1m[f 0801 15:30:40.631931 08 cuda_device_allocator.cc:31] Unable to alloc cuda device memory for size 236978176[m[m
**********
Async error was detected. To locate the async backtrace and get better error report, please rerun your code with two enviroment variables set:
>>> export JT_SYNC=1
>>> export tr

In [33]:
test_pred = predict(test_label, model_clip, preprocess, text_features, 1)
print(test_pred.shape)

100%|██████████| 3073/3073 [03:58<00:00, 12.89it/s]

(3073, 374)


In [34]:
top_k = 5
top_k_indices = np.argsort(test_pred, axis=1)[:, -top_k:]
top_k_indices = np.flip(top_k_indices, axis=1)

In [35]:
test_label[['Top1', 'Top2', 'Top3', 'Top4', 'Top5']] = top_k_indices

In [36]:
test_label.head(10)

,img_name,path,description,Top1,Top2,Top3,Top4,Top5
0,image_855.jpeg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_85...,None,365,363,303,349,326
1,image_2480.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_24...,None,145,143,151,202,162
2,image_1008.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_10...,None,165,151,164,160,241
3,image_1230.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_12...,None,79,80,115,61,303
4,image_1229.jpeg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_12...,None,246,247,244,329,362
5,image_2981.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_29...,None,252,351,276,280,363
6,image_1754.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_17...,None,157,160,151,241,226
7,image_1105.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_11...,None,358,250,360,363,371
8,image_319.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_31...,None,82,285,264,295,334
9,image_499.jpg,/mnt/workspace/JCLIP/Dataset/TestSetA/image_49...,None,160,164,241,226,157


In [37]:
columns_to_save = ['img_name', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5']
df_selected = test_label[columns_to_save]
df_selected.to_csv('../model_output/result_clip_ft.txt', sep=' ', index=False, header=False)